In [53]:
import pandas as pd
import re
import requests
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 1000)

## Seções eleitorais
Carrega a lista de seções eleitorais do estado de SP.

In [54]:
secoes_eleitorais = pd.read_csv('./secoes_eleitorais_sp.csv', encoding='latin1',
                               header=0, names=['zona', 'escola', 'endereco', 'cidade', 'secoes'],
                               index_col=False)
#apenas fernandopolist
secoes_eleitorais = secoes_eleitorais[secoes_eleitorais['cidade'] == 'FERNANDÓPOLIS'].drop(
    axis='columns', labels=['zona', 'cidade']).reset_index(drop=True)

In [55]:
def separa_numero_secoes(sessao):
    #grupos de seções separados por ;
    str_sessoes = sessao.split(';')
    #cada seção está no formato: de \d à \d
    todas_secoes = []
    for s in str_sessoes:
        secoes = re.findall('\d+', s)
        secoes = list(map(int, secoes))
        if len(secoes) == 2:
            secoes[1] = secoes[1] + 1
            secoes = list(range(*secoes))
        else:
            secoes = list(secoes)
        for s in secoes:
            todas_secoes.append(s)
    return todas_secoes
secoes_eleitorais['lista_secoes'] = secoes_eleitorais['secoes'].apply(separa_numero_secoes)

Renomeia as escolas

In [56]:
secoes_eleitorais['escola'] = secoes_eleitorais['escola'].map({
    'CENTRO EDUCACIONAL SESI  405': 'Sesi',
    'EE AFONSO CAFARO': 'Cafaro',
    'EE ARMELINDO FERRARI': 'Ferrari',
    'EE CARLOS BAROZZI': 'Barozi',
    'EE FERNANDO BARBOSA LIMA': 'Barbosa Lima',
    'EE JOAQUIM ANTONIO PEREIRA': 'JAP',
    'EE JOSE BELUCIO': 'Belucio',
    'EE LIBERO ALMEIDA SILVARES': 'EELAS',
    'EE PROF ANTONIO TANURI': 'Tanuri',
    'EE PROFA MARIA CONCEICAO APARECIDA BASSO': 'Brasitânia',
    'EE SATURNINO LEON ARROYO': 'Saturnino',
    'EMEF CEL FRANCISCO ARNALDO DA SILVA': 'Coronel',
    'EMEF JOAO GARCIA ANDREO': 'Andreo',
    'EMEF KOEI ARAKAKI': 'Koei Arakaki',
    'EMEF PEDRO MALAVAZZI': 'Pedro Malavazzi',
    'EMEF PROF  IVONETE AMARAL DA SILVA ROSA': 'Ivonete',
    'ESCOLA TÉCNICA ESTADUAL DE FERNANDÓPOLIS': 'ETEC'
})

Encontra as coordenadas geograficas da escola para plotar o gráfico depois.
Pesquisa por endereço através da OpenStreetMaps.

In [57]:
def pesquisa_endereco(endereco):
    try:
        # o número do endereço precisa estar na frente
        end, numero = re.findall('(^.+)(\,\s+\d+$)', endereco.strip())[0]
        numero = re.findall('\d+', numero)[0]
        r = requests.get(
            f'https://nominatim.openstreetmap.org/search?city=fernandopolis&street={numero}+{end}&format=json')
        res = r.json()
        if len(res) == 0:
            raise IndexError('Endereco não encontrado')
        return (r.json()[0]['lat'], r.json()[0]['lon'])
    except Exception as err:
        print(endereco, 'erro: ', err)
        return [0, 0]

In [58]:
lat_lon_df = pd.DataFrame(
    secoes_eleitorais['endereco'].apply(pesquisa_endereco).tolist(),
    columns=['lat', 'lon'])

AVENIDA RUBENS PADILHA MEATTO, 783 erro:  Endereco não encontrado
R REINALDO BERTOLINE, 300 erro:  Endereco não encontrado


In [59]:
# Escola Andreo e Pedro Malavazi, informa hard coded
lat_lon_df.iloc[13][['lat', 'lon']] = (-20.269411, -50.260) #Pedro Malavazi
lat_lon_df.iloc[14][['lat', 'lon']] = (-20.281775, -50.257071) #Andreo

In [60]:
secoes_eleitorais = pd.concat([secoes_eleitorais, lat_lon_df], axis='columns')

## Preparação dos dados de votação
1. Consome os datasets do primeiro e segundo turno;
2. Concatena
3. Join com as seções eleitorais

In [61]:
votos_turno1 = pd.read_csv('./2018_turno_1/votos_filtrados_fernandopolis.csv',
    encoding='latin1', sep=';', na_values=['#NULO', '#NULO#', '-1', -1])
votos_turno2 = pd.read_csv('./2018_turno_2/votos_filtrados_fernandopolis.csv',
    encoding='latin1', sep=';', na_values=['#NULO', '#NULO#', '-1', -1])
votos = pd.concat([votos_turno1, votos_turno2])

In [62]:
features = [
    'ANO_ELEICAO', 'NR_TURNO', 'NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO',
    'DS_CARGO_PERGUNTA', 'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'QT_APTOS',
    'QT_COMPARECIMENTO', 'QT_ABSTENCOES', 'DS_TIPO_VOTAVEL', 'NR_VOTAVEL',
    'NM_VOTAVEL', 'QT_VOTOS'
]
nome_features = [
    'ano', 'turno', 'zona_eleitoral', 'secao', 'local_votacao',
    'cargo', 'num_partido', 'sigla_partido', 'nome_partido', 'eleitores_aptos',
    'eleitores_compareceram', 'eleitores_absteram', 'voto_tipo', 'voto_numero',
    'voto_nome', 'voto_qtd'
]
nome_colunas = dict(zip(features, nome_features))
votos = votos[features].rename(columns=nome_colunas)

## Merge entre os 2 datasets
Encontra qual a seção eleitoral de cada voto

In [63]:
def encontra_secao(numero_secao):
    # procura a secao que o numero esta e retorna o indice
    return secoes_eleitorais[
        list(numero_secao in x for x in secoes_eleitorais['lista_secoes'])].index.values[0]
votos['idx_secao'] = votos['secao'].apply(encontra_secao)

In [64]:
df_eleicoes = pd.merge(votos, secoes_eleitorais, left_on='idx_secao', right_index=True)

In [65]:
#limpeza nos dados desnecessarios
df_eleicoes.drop(axis='columns', labels=['ano', 'secoes'], inplace=True)

In [66]:
# Apenas a primeira letra do nome dos candidatos maiúscula
df_eleicoes['voto_nome'] = df_eleicoes['voto_nome'].str.title()

## Salva o resultado para análise

In [69]:
df_eleicoes.to_csv('./eleicoes_final.csv', encoding='latin1', index=False)